In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = pd.read_csv('final_frame_pl_v4.csv')

data = data.drop(columns = ['B365H', 'B365D', 'B365A', 'Bet_diff', 'Bet_diff_bin', 'Bet_diff_std', 'Bet_winner'])
data = data.dropna()

In [3]:
data = data[data['FTR'] != 'D']
data['FTR'] = data['FTR'].replace({'H': 1, 'A': 0}).astype(float)  

In [4]:
data[['Pos_diff_std', 'Avg_pts_diff_std', 'Avg_goals_scored_diff_std', 'Form_diff_std', 'FTR']].dtypes

Pos_diff_std                 float64
Avg_pts_diff_std             float64
Avg_goals_scored_diff_std    float64
Form_diff_std                float64
FTR                          float64
dtype: object

In [5]:
features = ['Pos_diff_std', 'Avg_pts_diff_std', 'Avg_goals_scored_diff_std', 'Form_diff_std']
X = data[features]  
y = data['FTR']   

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predykcja i ocena
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7434620174346201
              precision    recall  f1-score   support

         0.0       0.69      0.62      0.65       624
         1.0       0.77      0.82      0.80       982

    accuracy                           0.74      1606
   macro avg       0.73      0.72      0.73      1606
weighted avg       0.74      0.74      0.74      1606



In [6]:
data.shape

(8026, 42)

In [7]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))

Accuracy: 0.7440846824408468
              precision    recall  f1-score   support

         0.0       0.69      0.62      0.65       624
         1.0       0.77      0.82      0.80       982

    accuracy                           0.74      1606
   macro avg       0.73      0.72      0.73      1606
weighted avg       0.74      0.74      0.74      1606



In [8]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

grid_search_xgb = RandomizedSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42), param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)

best_model_xgb = grid_search_xgb.best_estimator_

In [9]:
best_model.fit(X_train, y_train)

y_pred_best = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

Accuracy: 0.7540473225404732
              precision    recall  f1-score   support

         0.0       0.71      0.63      0.67       624
         1.0       0.78      0.83      0.81       982

    accuracy                           0.75      1606
   macro avg       0.74      0.73      0.74      1606
weighted avg       0.75      0.75      0.75      1606



In [10]:
best_model_xgb.fit(X_train, y_train)

y_pred_best_xgb = best_model_xgb.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_best_xgb))
print(classification_report(y_test, y_pred_best_xgb))

Accuracy: 0.7440846824408468
              precision    recall  f1-score   support

         0.0       0.69      0.62      0.65       624
         1.0       0.77      0.82      0.80       982

    accuracy                           0.74      1606
   macro avg       0.73      0.72      0.73      1606
weighted avg       0.74      0.74      0.74      1606



In [11]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

ensemble_model = VotingClassifier(estimators=[
    ('rf', best_model),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000))
], voting='soft')

ensemble_model.fit(X_train, y_train)
y_pred_ens_mod = ensemble_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_ens_mod))
print(classification_report(y_test, y_pred_ens_mod))

Accuracy: 0.7590286425902865
              precision    recall  f1-score   support

         0.0       0.71      0.64      0.67       624
         1.0       0.78      0.84      0.81       982

    accuracy                           0.76      1606
   macro avg       0.75      0.74      0.74      1606
weighted avg       0.76      0.76      0.76      1606

